# Importing libraries / checking GPU

In [ ]:
!nvidia-smi

In [2]:
import matplotlib.pyplot as plt # For visualizations
import numpy as np # For numerical computations
import tensorflow as tf # For machine learning models
import tensorflow_datasets as tfds # For loading datasets
import datetime # For date and time manipulation
from tensorflow.keras import layers, Model, models # For building Keras models
from tensorflow.keras.applications import EfficientNetB0 # Pre-trained model for image classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # For confusion matrix

tf.random.set_seed(42) # For reproducibility


# Load EuroSAT dataset with splits

In [3]:
# Define dataset splits
splits = ("train[:70%]", "train[70%:90%]", "train[90%:]")

# Load EuroSAT dataset with splits and metadata
(train, val, test), metadata = tfds.load(
    name="eurosat",  # Dataset name
    as_supervised=True,  # Load as (image, label) pairs
    split=splits,  # Training, validation, and test splits
    with_info=True  # Load dataset metadata
)

# Convert label indices to string labels
get_label_name = metadata.features["label"].int2str


# Preprocessing data

In [4]:
# Set parameters
batch_size = 32  # Samples per update
epochs = 10  # Training epochs
img_height = 64  # Image height
img_width = 64  # Image width
num_classes = metadata.features["label"].num_classes  # Unique class count

# Function to resize and normalize images
def resize_and_rescale(image, label):
    image = tf.cast(image, tf.float32)  # Convert to float32
    image = tf.image.resize(image, [img_height, img_width])  # Resize image
    image = image / 255.0  # Normalize to [0, 1]
    return image, label  # Return image and label

# Function to augment training images
def augment(image, label):
    image, label = resize_and_rescale(image, label)  # Resize and normalize
    image = tf.image.rot90(image)  # Rotate image 90 degrees
    image = tf.image.random_crop(image, size=[img_height, img_width, 3])  # Random crop
    return image, label  # Return augmented image and label


In [5]:
# Set auto-tuning for data pipeline
AUTOTUNE = tf.data.AUTOTUNE

# Shuffle, augment, batch, and prefetch the training dataset
train = (
    train.shuffle(1000)  # Shuffle the training data
    .map(augment, num_parallel_calls=AUTOTUNE)  # Apply augmentation
    .batch(batch_size)  # Batch the data
    .prefetch(AUTOTUNE)  # Prefetch for performance optimization
)

# Resize and batch the validation dataset
val = (
    val.map(resize_and_rescale, num_parallel_calls=AUTOTUNE)  # Resize and normalize
    .batch(batch_size)  # Batch the data
    .prefetch(AUTOTUNE)  # Prefetch for performance optimization
)

# Resize and batch the test dataset
test = (
    test.map(resize_and_rescale, num_parallel_calls=AUTOTUNE)  # Resize and normalize
    .batch(batch_size)  # Batch the data
    .prefetch(AUTOTUNE)  # Prefetch for performance optimization
)


# Load pre-trained EfficientNetB0 model with imagenet weights

In [ ]:
# Load the pre-trained EfficientNetB0 model without the top layer
base_model = EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights="imagenet"
)

# Build the complete model with additional classification layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(metadata.features['label'].num_classes, activation='softmax')
])

# Compile the model and show summary
model.summary()


# Fine-tuning the model

In [ ]:
# Set the base model to be trainable for fine-tuning
base_model.trainable = True

# Compile the model using the Adam optimizer with specified loss and metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Define the log directory for TensorBoard
log_dir = "logs" + datetime.datetime.now().strftime("/%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model using the training and validation datasets
history_fine = model.fit(
    train,  # Training data
    validation_data=val,  # Validation data
    epochs=20,  # Total epochs for training
    callbacks=[tensorboard_callback],  # Save best weights during training
)


# Evaluate the fine-tuned model on the test dataset

In [ ]:
test_loss, test_accuracy = model.evaluate(test)

In [ ]:
# Get predictions for the entire test dataset
y_pred = model.predict(test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predictions to class labels

# Get true labels for the entire test dataset
y_true = np.concatenate([y for x, y in test], axis=0)

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=metadata.features['label'].names)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()